# Dominik Adamczyk
## Laboratorium 3 - rozwiązania 

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.


In [26]:
def edit_distance(str1, str2):
    m = len(str1)
    n = len(str2)
    dp = [[0 for j in range(n+1)] for i in range(m+1)]
    
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp

2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:

In [27]:
def edit_distance_steps(str1, str2):
    m = len(str1)
    n = len(str2)
    dp = edit_distance(str1, str2)
    distance = dp[m][n]
    edits = []
    i, j = m, n
    while i > 0 or j > 0:
        if i > 0 and dp[i-1][j]+1 == dp[i][j]:
            edits.append(("DELETE", str1[i-1], j))
            i -= 1
        elif j > 0 and dp[i][j-1]+1 == dp[i][j]:
            edits.append(("INSERT", str2[j-1], j-1))
            j -= 1
        elif i > 0 and j > 0 and dp[i-1][j-1]+1 == dp[i][j]:
            edits.append(("SUBSTITUTE", str1[i-1], str2[j-1], j-1))
            i -= 1
            j -= 1
        else:
            i -= 1
            j -= 1
            
    edits.reverse()
    return distance, edits

def visualize_edit_distance(str1, str2):
    distance, edits = edit_distance_steps(str1, str2)
    i, j = 0, 0
    # for edit in edits:
    #     if edit == "SUBSTITUTE":
    #         
    print(f"[INITIAL] {str1}")
    for edit in edits:
        if edit[0] == 'INSERT':
            new_str = str1[:edit[2]] + edit[1] + str1[edit[2]:]
            print(f'[INSERT] Letter \'{edit[1]}\' on position {edit[2]}')
        elif edit[0] == 'DELETE':
            new_str = str1[:edit[2]] + str1[edit[2] + 1:]
            print(f'[DELETE] Letter \'{edit[1]}\' on position {edit[2]}')
        else:
            new_str = str1[:edit[3]] + edit[2] + str1[edit[3] + 1:]
            # new_str = str1[:i] + str2[i] + str1[i + 1:]
            print(f"[SUBSTITUTE] '{edit[1]}' -> '{edit[2]}' on position {edit[3]}")
        print(f">>> {str1} --> {new_str}")
        str1 = new_str
    print(f"[FINISH] {str1}")

3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
    1. los - kloc
    2. Łódź - Lodz
    3. kwintesencja - quintessence
    4. ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [15]:
visualize_edit_distance("los", "kloc")

[INITIAL] los
[INSERT] Letter 'k' on position 0
>>> los --> klos
[SUBSTITUTE] 's' -> 'c' on position 3
>>> klos --> kloc
[FINISH] kloc


In [16]:
visualize_edit_distance("Łódź", "Lodz")

[INITIAL] Łódź
[SUBSTITUTE] 'Ł' -> 'L' on position 0
>>> Łódź --> Lódź
[SUBSTITUTE] 'ó' -> 'o' on position 1
>>> Lódź --> Lodź
[SUBSTITUTE] 'ź' -> 'z' on position 3
>>> Lodź --> Lodz
[FINISH] Lodz


In [17]:
visualize_edit_distance("kwintesencja", "quintessence")

[INITIAL] kwintesencja
[SUBSTITUTE] 'k' -> 'q' on position 0
>>> kwintesencja --> qwintesencja
[SUBSTITUTE] 'w' -> 'u' on position 1
>>> qwintesencja --> quintesencja
[INSERT] Letter 's' on position 7
>>> quintesencja --> quintessencja
[SUBSTITUTE] 'j' -> 'e' on position 11
>>> quintessencja --> quintessencea
[DELETE] Letter 'a' on position 12
>>> quintessencea --> quintessence
[FINISH] quintessence


In [18]:
visualize_edit_distance("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

[INITIAL] ATGAATCTTACCGCCTCG
[SUBSTITUTE] 'A' -> 'G' on position 4
>>> ATGAATCTTACCGCCTCG --> ATGAGTCTTACCGCCTCG
[SUBSTITUTE] 'T' -> 'G' on position 5
>>> ATGAGTCTTACCGCCTCG --> ATGAGGCTTACCGCCTCG
[INSERT] Letter 'C' on position 8
>>> ATGAGGCTTACCGCCTCG --> ATGAGGCTCTACCGCCTCG
[SUBSTITUTE] 'A' -> 'G' on position 10
>>> ATGAGGCTCTACCGCCTCG --> ATGAGGCTCTGCCGCCTCG
[INSERT] Letter 'G' on position 11
>>> ATGAGGCTCTGCCGCCTCG --> ATGAGGCTCTGGCCGCCTCG
[DELETE] Letter 'G' on position 14
>>> ATGAGGCTCTGGCCGCCTCG --> ATGAGGCTCTGGCCCCTCG
[DELETE] Letter 'C' on position 17
>>> ATGAGGCTCTGGCCCCTCG --> ATGAGGCTCTGGCCCCTG
[FINISH] ATGAGGCTCTGGCCCCTG


4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.


In [19]:
def lcs(str1, str2):
    m = len(str1)
    n = len(str2)
    lcs_table = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i-1] == str2[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i][j-1], lcs_table[i-1][j])
    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if str1[i-1] == str2[j-1]:
            lcs.append(str1[i-1])
            i -= 1
            j -= 1
        elif lcs_table[i-1][j] > lcs_table[i][j-1]:
            i -= 1
        else:
            j -= 1

    return lcs_table[-1][-1], lcs[::-1]

def longest_common_substring(str1, str2):
    m = len(str1)
    n = len(str2)
    lcs_table = [[0] * (n + 1) for i in range(m + 1)]
    longest_substring = ""
    longest_length = 0
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i-1] == str2[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
                if lcs_table[i][j] > longest_length:
                    longest_length = lcs_table[i][j]
                    longest_substring = str1[i-longest_length:i]
            else:
                lcs_table[i][j] = 0
    return longest_substring

In [20]:
lcs('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG')[0]

13

In [21]:
longest_common_substring('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG')

'ATGA'

5. Korzystając z gotowego tokenizera (np spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.


In [22]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

nlp = Polish()
tokenizer = Tokenizer(nlp.vocab);

with open("romeo-i-julia-700.txt", "r", encoding='utf-8') as f:
    RomeoiJulia = f.read()

tokens = tokenizer(RomeoiJulia)

6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.


In [23]:
import random

def del_tokens(tokens, precentToDel):
    return [token for token in tokens if random.randrange(100) > precentToDel]

Romeoi = del_tokens(tokens, 3)
iJulia = del_tokens(tokens, 3)

with open("Romeoi.txt", "w", encoding='utf-8') as f:
    for token in Romeoi:
        f.write(token.text_with_ws)
with open("iJulia.txt", "w", encoding="utf-8") as f:
    for token in iJulia:
        f.write(token.text_with_ws)

7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu.


In [24]:
print(f"Longest common subsequence: {lcs(Romeoi, iJulia)[0]}")

print(f"Longest common substring: {len(longest_common_substring(Romeoi, iJulia))}")

Longest common subsequence: 2082
Longest common substring: 93


8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

In [28]:
def file_diff(f1, f2):
    with open(f1, "r", encoding='utf-8') as f:
        first = f.read()
    with open(f2, "r", encoding='utf-8') as f:
        second = f.read()
    first = first.split("\n")
    second = second.split("\n")
    _, common = lcs(first, second);
    first = [a for a in enumerate(first)]
    second = [a for a in enumerate(second)]
    i = j = k = 0;
    while k < len(common):
        if common[k] == first[i][1] and common [k] == second[j][1]:
            k += 1
            i += 1
            j += 1
            continue
        if common[k] != first[i][1]:
            print(f"> ({first[i][0]})  {first[i][1]}")
            i += 1
        if common[k] != second[j][1]:
            print(f"< ({second[j][0]})  {second[j][1]}")
            j += 1
    while i < len(first):
        print(f"> ({first[i][0]})  {first[i][1]}")
        i += 1
    while j < len(second):
        print(f"> ({second[j][0]})  {second[j][1]}")
        j += 1


9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [29]:
file_diff("Romeoi.txt", "iJulia.txt")

> (2)  Romeo i Julia
< (2)  Romeo Julia
> (5)  ISBN 978-83-288-2903-9OSOBY:
< (5)  ISBN 978-83-288-2903-9
< (6)  
< (7)  
< (8)  
< (9)  OSOBY:
> (7)   * PARYS — Weroneńczyk szlachetnego rodu, krewny 
< (11)   * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
> (8)   * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< (12)   * MONTEKI, KAPULET — naczelnicy dwóch nieprzyjaznych sobie
> (9)   * STARZEC — stryjeczny brat Kapuleta
< (13)   * — stryjeczny brat Kapuleta
> (10)   * ROMEO — syn Montekiego
< (14)   * ROMEO — syn 
> (11)   * MERKUCJO — krewny * BENWOLIO synowiec Montekiego
< (15)   * MERKUCJO — krewny księcia
< (16)   * BENWOLIO — synowiec Montekiego
> (13)   — ojciec franciszkanin
< (18)   * LAURENTY — ojciec franciszkanin
> (27)   * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne 
< (32)   * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż